#### Import Libraries

In [36]:
import pandas as pd
# from goodreads import client
# from isbnlib import desc, clean, is_isbn10
import numpy as np
# from bs4 import BeautifulSoup
# import nltk
# import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import gensim
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

#### Import Ratings Data

In [2]:
ratings = pd.read_csv('../ratings.csv')

In [3]:
ratings.shape

(5976479, 3)

In [4]:
ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [5]:
book_ids = ratings.book_id.unique() # 10000 unique book ids

In [6]:
len(book_ids)

10000

#### Import Book Metadata

In [7]:
book_info = pd.read_csv('../books.csv')

In [8]:
book_info.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [9]:
book_info.columns

Index(['book_id', 'goodreads_book_id', 'best_book_id', 'work_id',
       'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year',
       'original_title', 'title', 'language_code', 'average_rating',
       'ratings_count', 'work_ratings_count', 'work_text_reviews_count',
       'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5',
       'image_url', 'small_image_url'],
      dtype='object')

#### Merge in Relevant Metadata

In [10]:
ratings = ratings.merge(right=book_info[['book_id', 'isbn', 'authors', 'original_publication_year', 
                                         'original_title']], on='book_id')

#### Get Book Summaries

In [ ]:
# gc = client.GoodreadsClient('l2td8Sa3lHfqooxdH5w','i5ReUzTEBrPfoEo1wKCc3VHUeU8KodEzUDwTBP0')

In [ ]:
# uses the goodreads api to get a summary and cleans it with beautifulsoup

# def get_clean_summary(b_id, ratings):
#     isbn = ratings.isbn[ratings['book_id'] == b_id].iloc[0]
#     if isinstance(isbn, str):
#         while len(isbn) < 10:
#             isbn = '0' + isbn
#         try:
#             item = gc.book(isbn=isbn)
#         except:
#             try:
#                 isbn = isbn[1:]
#                 item = gc.book(isbn=isbn)
#             except:
#                 return None
#         desc = item.description
#         if desc is not None:
#             clean = BeautifulSoup(desc, 'lxml').text
#             return clean
#         else:
#             return None
#     else:
#         return None

In [ ]:
# summaries = []
# count = 0
# for book_id in book_ids:
#     count += 1
#     if count % 100 == 0:
#         print(count)
#     new_dict = {'book_id': book_id}
#     new_dict['description'] = get_clean_summary(book_id, ratings)
#     summaries.append(new_dict)

In [ ]:
# summaries_df = pd.DataFrame(summaries)

In [ ]:
# gets a summary using isbnlib (for books with no goodreads summary)

# def get_isbnlib_desc(row):
#     isbn = ratings.isbn[ratings['book_id'] == row['book_id']].iloc[0]
#     if isinstance(isbn, str):
#         if is_isbn10(isbn) == False and len(isbn) < 10:
#             isbn = '0'*(10-len(isbn)) + isbn
#         try:
#             description = desc(isbn)
#         except:
#             return None
#         if description:
#             clean = BeautifulSoup(description, 'lxml').text
#             return clean
#         else:
#             return None
#     elif np.isnan(isbn):
#         return None
#     else:
#         print(isbn)

In [ ]:
# summaries_df[summaries_df.description.isnull()].merge(book_info, on='book_id')['book_id']

In [ ]:
# for i in range(len(summaries_df)):
#     if i % 200 == 0:
#         print(i)
#     if isinstance(summaries_df['description'].iloc[i], str):
#         pass
#     elif np.isnan(summaries_df['description'].iloc[i]):
#         summaries_df['description'].iloc[i] = get_isbnlib_desc(summaries_df.iloc[i])

In [ ]:
# bring in the original title again for merging purposes

# summaries_df['original_title'] = summaries_df.apply(lambda x: ratings[ratings.book_id == x.book_id].iloc[0]['original_title'], axis=1)

In [15]:
# summaries_df.to_csv('summaries.csv')

In [11]:
summaries_df = pd.read_csv('summaries.csv')

In [12]:
summaries_df.shape

(10000, 4)

In [13]:
summaries_df.head()

,Unnamed: 0,book_id,description,original_title
0,0,258,"The international literary sensation, about a ...",La sombra del viento
1,1,4081,"Tom Wolfe, the master social novelist of our t...",NaN
2,2,260,NaN,How to Win Friends and Influence People
3,3,9296,As charming performers who skillfully reflect ...,Das Drama des begabten Kindes und die Suche na...
4,4,2318,The incredible national bestseller that is cha...,The Millionaire Next Door: The Surprising Secr...


#### NLP on summaries

In [ ]:
# get summaries with a description

# valid_summaries = pd.DataFrame(summaries_df[summaries_df.description.notnull()][['original_title', 'description']])

In [ ]:
# def get_tokens(row):
#     tokens = nltk.word_tokenize(row['description'])
#     no_punc = [i for i in tokens if i not in list(string.punctuation)]
#     return no_punc

In [ ]:
# tokenize

# valid_summaries['tokens'] = valid_summaries.apply(lambda row: get_tokens(row), axis=1)

In [ ]:
# valid_summaries.to_pickle('valid_summaries.p')

In [14]:
valid_summaries = pd.read_pickle('valid_summaries.p')

In [15]:
valid_summaries.shape

(9106, 3)

In [16]:
valid_summaries.head()

,original_title,description,tokens
0,La sombra del viento,"The international literary sensation, about a ...","[The, international, literary, sensation, abou..."
1,NaN,"Tom Wolfe, the master social novelist of our t...","[Tom, Wolfe, the, master, social, novelist, of..."
3,Das Drama des begabten Kindes und die Suche na...,As charming performers who skillfully reflect ...,"[As, charming, performers, who, skillfully, re..."
4,The Millionaire Next Door: The Surprising Secr...,The incredible national bestseller that is cha...,"[The, incredible, national, bestseller, that, ..."
5,The Da Vinci Code,A fascinating and absorbing thriller -- perfec...,"[A, fascinating, and, absorbing, thriller, --,..."


#### tf-idf

In [ ]:
# tfidf descriptions

# tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,3), min_df=2)
# ngrams_tfidf = tfidf.fit_transform(valid_summaries.description)
# tfidf_array = ngrams_tfidf.toarray()

In [ ]:
# tfidf_array.shape

In [ ]:
# scale and pca for 20 components

# pca = make_pipeline(StandardScaler(), PCA(n_components=20))
# tfidf_pca = pca.fit_transform(tfidf_array)

In [ ]:
# tfidf_pca.shape

In [ ]:
# tfidf_pca_df = pd.DataFrame(tfidf_pca)
# tfidf_pca_df.to_pickle('tfidf_pca.p')

In [17]:
tfidf_pca_df = pd.read_pickle('tfidf_pca.p')

In [29]:
tfidf_pca_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,-0.243430,-0.288574,-0.235490,-0.030586,-0.412304,-0.092544,-0.292133,-0.202417,-0.196980,-0.224375,-0.056896,-0.449349,-0.164316,-0.288940
1,-0.308547,-0.288131,-0.474792,0.096718,-0.311331,-0.297464,-0.357227,0.005336,-0.166258,-0.298781,-0.476267,0.025779,-0.286845,0.353179,-0.182561,-0.166128,-0.147104,-0.042784,0.220591,-0.402107
2,-0.240595,-0.186942,-0.019210,-0.341454,-0.223834,-0.308025,-0.311226,-0.154156,-0.214751,-0.281479,-0.141648,-0.012292,-0.268262,-0.221511,-0.077561,-0.154002,0.077414,-0.341870,-0.092560,-0.309710
3,-0.262850,-0.283931,-0.338522,-0.293817,-0.228251,-0.293628,-0.268385,-0.141414,-0.178740,-0.207532,-0.306377,-0.133941,-0.280633,-0.200243,-0.269785,-0.163894,-0.008157,-0.275194,0.002126,-0.314186
4,-0.327019,-0.306978,-0.518494,-0.066839,-0.208867,-0.361792,-0.472664,-0.250100,-0.279933,-0.310138,-0.384977,-0.146799,-0.459027,-0.206520,-0.208521,-0.110003,-0.223444,-0.947658,0.963800,-1.300039


In [18]:
tfidf_df = pd.concat([valid_summaries.reset_index(), tfidf_pca_df], axis=1)

#### Pre-trained embeddings

In [37]:
# w = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
# vect_w2v = CountVectorizer(vocabulary=w.index2word)
# vect_w2v.fit(valid_summaries['description'])
# docs = vect_w2v.inverse_transform(vect_w2v.transform(valid_summaries['description']))

In [53]:
# vecs = np.zeros(shape=(len(docs), 300))
# for i in range(len(docs)):
#     try:
#         vecs[i,:] = np.mean(w[docs[i]], axis=0)
#     except:
#         vecs[i,:] = [0.1]*300

In [54]:
# vecs.shape

(9106, 300)

In [55]:
# vecs_df = pd.DataFrame(vecs)

In [20]:
# vecs_df.shape

(9106, 300)

In [57]:
# vecs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.079045,0.046978,0.024757,0.072995,-0.031768,0.023041,0.048417,-0.074903,0.084486,0.067970,...,-0.058866,0.012489,-0.106315,0.025959,-0.057459,-0.018482,0.013247,-0.032685,0.022442,0.036078
1,0.036860,0.054040,0.024671,0.090106,-0.009635,-0.007365,0.041575,-0.086984,0.043074,0.065224,...,-0.058753,-0.003511,-0.128945,-0.012384,-0.048017,-0.034056,0.036770,-0.044904,0.002748,0.042575
2,0.067710,0.056081,0.005387,0.102136,-0.091895,0.015939,0.064989,-0.068714,0.078365,0.043722,...,-0.079463,0.051270,-0.098093,-0.033289,-0.080216,-0.014591,0.027747,-0.023640,0.060063,-0.016849
3,0.049386,0.020477,0.009153,0.078558,-0.046035,-0.001834,0.023504,-0.073450,0.063702,0.077257,...,-0.054595,0.052787,-0.099926,0.021063,-0.031636,-0.020733,0.004382,-0.055060,0.032753,-0.057663
4,0.060657,0.036889,0.025680,0.071606,-0.039362,0.027025,0.050223,-0.059705,0.062407,0.064980,...,-0.052157,0.006440,-0.077963,0.037847,-0.063216,-0.054680,-0.006997,-0.057613,0.015098,0.025199


In [58]:
# scale and pca for 20 components

# pca_vecs = make_pipeline(StandardScaler(), PCA(n_components=20))
# vecs_pca = pca_vecs.fit_transform(vecs_df)

In [59]:
# vecs_pca_df = pd.DataFrame(vecs_pca)

In [60]:
# vecs_pca_df.shape

(9106, 20)

In [61]:
# vecs_pca_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.238178,3.488654,2.057652,1.655820,-0.898556,-0.156045,1.014764,2.461330,0.106843,-1.163905,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.339720,-0.092151,-1.892633,0.210597
1,3.107542,4.579044,2.028579,3.259026,-1.140304,-1.034933,-0.406780,0.650010,-0.981423,2.163757,-4.709953,-2.067705,-2.183916,-1.984647,-2.578708,0.348261,0.182132,0.210395,0.877238,1.475320
2,-2.882872,4.532057,-3.088042,-0.568094,-3.501648,-7.068968,0.391202,-1.226969,0.636296,2.153693,1.545850,-0.368972,-2.457852,-0.854603,3.318846,0.407702,2.294161,0.442469,3.453820,-0.043973
3,-4.784111,-4.102846,-7.393620,-1.620640,-0.759388,0.752658,-1.146854,-0.860671,0.177649,-1.277452,-2.150216,2.403672,-1.876621,3.148182,-1.488503,-4.015573,-0.856176,-0.201446,-0.743828,1.455489
4,3.260447,2.936169,1.241696,-1.052145,0.094371,2.022580,0.829372,4.161351,-1.718266,0.733636,2.140828,-1.573132,1.669294,1.016045,-0.888446,-2.065023,-2.634372,-0.774600,-1.436559,1.655652


In [77]:
# vecs_pca_df.to_pickle('vecs.p')

In [19]:
vecs_df = pd.read_pickle('vecs.p')

#### Combine NLP Features

In [62]:
# nlp_df = pd.concat([tfidf_df, vecs_pca_df], axis=1)

In [63]:
# nlp_df.shape

(9106, 44)

In [64]:
# nlp_df.head()

,index,original_title,description,tokens,0,1,2,3,4,5,...,10,11,12,13,14,15,16,17,18,19
0,0,La sombra del viento,"The international literary sensation, about a ...","[The, international, literary, sensation, abou...",-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.339720,-0.092151,-1.892633,0.210597
1,1,NaN,"Tom Wolfe, the master social novelist of our t...","[Tom, Wolfe, the, master, social, novelist, of...",-0.308547,-0.288131,-0.474792,0.096718,-0.311331,-0.297464,...,-4.709953,-2.067705,-2.183916,-1.984647,-2.578708,0.348261,0.182132,0.210395,0.877238,1.475320
2,3,Das Drama des begabten Kindes und die Suche na...,As charming performers who skillfully reflect ...,"[As, charming, performers, who, skillfully, re...",-0.240595,-0.186942,-0.019210,-0.341454,-0.223834,-0.308025,...,1.545850,-0.368972,-2.457852,-0.854603,3.318846,0.407702,2.294161,0.442469,3.453820,-0.043973
3,4,The Millionaire Next Door: The Surprising Secr...,The incredible national bestseller that is cha...,"[The, incredible, national, bestseller, that, ...",-0.262850,-0.283931,-0.338522,-0.293817,-0.228251,-0.293628,...,-2.150216,2.403672,-1.876621,3.148182,-1.488503,-4.015573,-0.856176,-0.201446,-0.743828,1.455489
4,5,The Da Vinci Code,A fascinating and absorbing thriller -- perfec...,"[A, fascinating, and, absorbing, thriller, --,...",-0.327019,-0.306978,-0.518494,-0.066839,-0.208867,-0.361792,...,2.140828,-1.573132,1.669294,1.016045,-0.888446,-2.065023,-2.634372,-0.774600,-1.436559,1.655652


#### Get Book IDs, Handle Missing Values

In [65]:
# merge summary information into df with ALL books

# books_with_summaries = summaries_df.merge(nlp_df, on=['description','original_title'], how='left')

In [66]:
# books_with_summaries.shape

(10000, 46)

In [67]:
# books_with_summaries.head()

,Unnamed: 0,book_id,description,original_title,index,tokens,0,1,2,3,...,10,11,12,13,14,15,16,17,18,19
0,0,258,"The international literary sensation, about a ...",La sombra del viento,0.0,"[The, international, literary, sensation, abou...",-0.230597,-0.250082,-0.377935,-0.266002,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.339720,-0.092151,-1.892633,0.210597
1,1,4081,"Tom Wolfe, the master social novelist of our t...",NaN,1.0,"[Tom, Wolfe, the, master, social, novelist, of...",-0.308547,-0.288131,-0.474792,0.096718,...,-4.709953,-2.067705,-2.183916,-1.984647,-2.578708,0.348261,0.182132,0.210395,0.877238,1.475320
2,2,260,NaN,How to Win Friends and Influence People,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,9296,As charming performers who skillfully reflect ...,Das Drama des begabten Kindes und die Suche na...,3.0,"[As, charming, performers, who, skillfully, re...",-0.240595,-0.186942,-0.019210,-0.341454,...,1.545850,-0.368972,-2.457852,-0.854603,3.318846,0.407702,2.294161,0.442469,3.453820,-0.043973
4,4,2318,The incredible national bestseller that is cha...,The Millionaire Next Door: The Surprising Secr...,4.0,"[The, incredible, national, bestseller, that, ...",-0.262850,-0.283931,-0.338522,-0.293817,...,-2.150216,2.403672,-1.876621,3.148182,-1.488503,-4.015573,-0.856176,-0.201446,-0.743828,1.455489


In [73]:
# give 0.1 for all missing values

# numpy_version = np.array(books_with_summaries.iloc[:,6:])
# numpy_version[np.isnan(numpy_version)]=0.1
# numpy_pd = pd.DataFrame(numpy_version)

In [74]:
# books_with_summaries_new = pd.concat([books_with_summaries.book_id, numpy_pd], axis=1)

In [75]:
# books_with_summaries_new.head()

,book_id,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,258,-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,-0.243430,-0.288574,-0.235490,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.339720,-0.092151,-1.892633,0.210597
1,4081,-0.308547,-0.288131,-0.474792,0.096718,-0.311331,-0.297464,-0.357227,0.005336,-0.166258,...,-4.709953,-2.067705,-2.183916,-1.984647,-2.578708,0.348261,0.182132,0.210395,0.877238,1.475320
2,260,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
3,9296,-0.240595,-0.186942,-0.019210,-0.341454,-0.223834,-0.308025,-0.311226,-0.154156,-0.214751,...,1.545850,-0.368972,-2.457852,-0.854603,3.318846,0.407702,2.294161,0.442469,3.453820,-0.043973
4,2318,-0.262850,-0.283931,-0.338522,-0.293817,-0.228251,-0.293628,-0.268385,-0.141414,-0.178740,...,-2.150216,2.403672,-1.876621,3.148182,-1.488503,-4.015573,-0.856176,-0.201446,-0.743828,1.455489


In [76]:
# books_with_summaries_new.to_pickle('books_with_summaries.p')

In [87]:
books_with_summaries = pd.read_pickle('books_with_summaries.p')

In [88]:
books_with_summaries.shape

(10000, 41)

In [89]:
books_with_summaries.head()

,book_id,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,258,-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,-0.243430,-0.288574,-0.235490,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.339720,-0.092151,-1.892633,0.210597
1,4081,-0.308547,-0.288131,-0.474792,0.096718,-0.311331,-0.297464,-0.357227,0.005336,-0.166258,...,-4.709953,-2.067705,-2.183916,-1.984647,-2.578708,0.348261,0.182132,0.210395,0.877238,1.475320
2,260,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
3,9296,-0.240595,-0.186942,-0.019210,-0.341454,-0.223834,-0.308025,-0.311226,-0.154156,-0.214751,...,1.545850,-0.368972,-2.457852,-0.854603,3.318846,0.407702,2.294161,0.442469,3.453820,-0.043973
4,2318,-0.262850,-0.283931,-0.338522,-0.293817,-0.228251,-0.293628,-0.268385,-0.141414,-0.178740,...,-2.150216,2.403672,-1.876621,3.148182,-1.488503,-4.015573,-0.856176,-0.201446,-0.743828,1.455489


#### Merge Summaries into Ratings Data

In [90]:
ratings_nlp = ratings.merge(right=books_with_summaries, on='book_id', how='left')

In [91]:
ratings_nlp.shape

(5976479, 47)

In [92]:
ratings_nlp.head()

,user_id,book_id,rating,isbn,authors,original_publication_year,original_title,0,1,2,...,30,31,32,33,34,35,36,37,38,39
0,1,258,5,143034901,"Carlos Ruiz Zafón, Lucia Graves",2001.0,La sombra del viento,-0.230597,-0.250082,-0.377935,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597
1,11,258,3,143034901,"Carlos Ruiz Zafón, Lucia Graves",2001.0,La sombra del viento,-0.230597,-0.250082,-0.377935,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597
2,143,258,4,143034901,"Carlos Ruiz Zafón, Lucia Graves",2001.0,La sombra del viento,-0.230597,-0.250082,-0.377935,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597
3,242,258,5,143034901,"Carlos Ruiz Zafón, Lucia Graves",2001.0,La sombra del viento,-0.230597,-0.250082,-0.377935,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597
4,325,258,4,143034901,"Carlos Ruiz Zafón, Lucia Graves",2001.0,La sombra del viento,-0.230597,-0.250082,-0.377935,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597


In [93]:
ratings_nlp.tail()

,user_id,book_id,rating,isbn,authors,original_publication_year,original_title,0,1,2,...,30,31,32,33,34,35,36,37,38,39
5976474,36773,9548,5,NaN,L.J. Shen,2016.0,Vicious,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
5976475,50673,9548,4,NaN,L.J. Shen,2016.0,Vicious,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
5976476,45213,9548,3,NaN,L.J. Shen,2016.0,Vicious,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
5976477,12872,9548,4,NaN,L.J. Shen,2016.0,Vicious,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
5976478,46353,9548,5,NaN,L.J. Shen,2016.0,Vicious,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [94]:
just_nlp = ratings_nlp.iloc[:,-40:]

In [95]:
just_nlp.shape

(5976479, 40)

In [96]:
just_nlp

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,-0.24343,-0.288574,-0.23549,-0.030586,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597
1,-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,-0.24343,-0.288574,-0.23549,-0.030586,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597
2,-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,-0.24343,-0.288574,-0.23549,-0.030586,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597
3,-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,-0.24343,-0.288574,-0.23549,-0.030586,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597
4,-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,-0.24343,-0.288574,-0.23549,-0.030586,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597
5,-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,-0.24343,-0.288574,-0.23549,-0.030586,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597
6,-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,-0.24343,-0.288574,-0.23549,-0.030586,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597
7,-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,-0.24343,-0.288574,-0.23549,-0.030586,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597
8,-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,-0.24343,-0.288574,-0.23549,-0.030586,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597
9,-0.230597,-0.250082,-0.377935,-0.266002,-0.304168,-0.343982,-0.24343,-0.288574,-0.23549,-0.030586,...,2.251627,-0.114333,1.656405,1.347895,0.134338,-0.168178,0.33972,-0.092151,-1.892633,0.210597


In [97]:
just_nlp.to_csv('nlp_features.csv', header=False)